# 🧠  Economic Context & Synthesis (EDA + Modeling)
## 🎯 Mục tiêu
Phân tích các **biến kinh tế vĩ mô** ảnh hưởng đến việc **khách hàng có gửi tiền (y)** hay không.  
Sau đó, xây dựng các mô hình dự đoán với **PySpark**, xử lý dữ liệu **mất cân bằng** bằng **SMOTE**,  và đánh giá mô hình bằng **Cross-validation**.


In [ ]:
# Import các thư viện cần thiết
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



# Tạo SparkSession
spark = SparkSession.builder.appName("Hao_Economic_Context_Synthesis").getOrCreate()

# Đọc dữ liệu
data = spark.read.csv("../data/bank-additional/bank-additional-full.csv", header=True, sep=';', inferSchema=True)
print("Số dòng:", data.count())
data.printSchema()


Số dòng: 41188
root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- y: string (nullable = true)



In [4]:
data.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

## 🔍 2️⃣ EDA - Phân tích ảnh hưởng của các biến kinh tế vĩ mô
Ở đây ta tập trung vào 5 biến vĩ mô:
- `emp.var.rate` – tỷ lệ biến động việc làm  
- `cons.price.idx` – chỉ số giá tiêu dùng  
- `cons.conf.idx` – chỉ số niềm tin người tiêu dùng  
- `euribor3m` – lãi suất EURIBOR 3 tháng  
- `nr.employed` – số người có việc làm  

Ta sẽ xem xét **phân phối**, **tương quan** và **ảnh hưởng** của chúng tới biến mục tiêu `y`.


In [8]:
# Đổi tên cột để tránh lỗi có dấu chấm
data = (data
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

macro_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]

In [ ]:
from pyspark.sql.functions import col, mean, stddev, min as _min, max as _max, count, corr
from pyspark.sql.types import FloatType
# === ĐỔI TÊN CỘT CHO HỢP LỆ (nếu chưa làm) ===
data = (data
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

# Danh sách các biến vĩ mô
macro_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]

# --- 1️⃣ Thống kê mô tả cơ bản ---
print("=== 1️⃣ Thống kê mô tả cơ bản ===")
data.select(macro_cols).describe().show(truncate=False)

# --- 2️⃣ Tỷ lệ các lớp mục tiêu (y) ---
print("=== 2️⃣ Tỷ lệ phân bố biến mục tiêu y ===")
total_count = data.count()
data.groupBy("y").agg(
    count("*").alias("count")
).withColumn(
    "percentage", (col("count") / total_count * 100)
).show()

# --- 3️⃣ Trung bình các biến vĩ mô theo từng giá trị y ---
print("=== 3️⃣ Trung bình các biến vĩ mô theo từng giá trị y ===")
agg_exprs = [mean(c).alias(f"avg_{c}") for c in macro_cols]
data.groupBy("y").agg(*agg_exprs).show(truncate=False)




=== 1️⃣ Thống kê mô tả cơ bản ===
+-------+-------------------+------------------+------------------+------------------+-----------------+
|summary|emp_var_rate       |cons_price_idx    |cons_conf_idx     |euribor3m         |nr_employed      |
+-------+-------------------+------------------+------------------+------------------+-----------------+
|count  |41188              |41188             |41188             |41188             |41188            |
|mean   |0.08188550063178392|93.57566436828918 |-40.50260027191787|3.6212908128585366|5167.035910944004|
|stddev |1.57095974051703   |0.5788400489541355|4.628197856174595 |1.7344474048512557|72.25152766825924|
|min    |-3.4               |92.201            |-50.8             |0.634             |4963.6           |
|max    |1.4                |94.767            |-26.9             |5.045             |5228.1           |
+-------+-------------------+------------------+------------------+------------------+-----------------+

=== 2️⃣ Tỷ lệ phân b

In [14]:
# --- 4️⃣ Ma trận tương quan giữa các biến vĩ mô ---
print("=== 4️⃣ Ma trận tương quan giữa các biến vĩ mô ===")
corr_matrix = []
for c1 in macro_cols:
    row = []
    for c2 in macro_cols:
        try:
            val = data.select(corr(c1, c2).alias("corr")).collect()[0]["corr"]
        except Exception as e:
            val = None
        row.append(val)
    corr_matrix.append((c1, *row))

spark.createDataFrame(corr_matrix, ["Feature"] + macro_cols).show(truncate=False)

=== 4️⃣ Ma trận tương quan giữa các biến vĩ mô ===
+--------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|Feature       |emp_var_rate       |cons_price_idx     |cons_conf_idx       |euribor3m          |nr_employed        |
+--------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|emp_var_rate  |1.0                |0.775334170834832  |0.19604126813197284 |0.9722446711516147 |0.9069701012560353 |
|cons_price_idx|0.775334170834832  |1.0                |0.058986181748833216|0.688230107037495  |0.5220339770130168 |
|cons_conf_idx |0.19604126813197287|0.05898618174883324|1.0                 |0.27768621966375506|0.10051343183753894|
|euribor3m     |0.9722446711516147 |0.6882301070374951 |0.27768621966375506 |1.0                |0.9451544313982513 |
|nr_employed   |0.9069701012560353 |0.5220339770130166 |0.10051343183753896 |0.9451544313982513 |1.0                |
+----

- Nhóm biến emp.var.rate, euribor3m, nr.employed có thể được rút gọn hoặc chọn 1–2 đại diện.

- Các biến có tương quan âm với y

- Khi các chỉ số kinh tế này tăng (biểu hiện cho tình hình kinh tế tốt hơn), khả năng khách hàng đồng ý gửi tiền (y=1) lại giảm.

| Biến vĩ mô (`feature`) | Ý nghĩa kinh tế                 | Xu hướng khi biến tăng             | Ảnh hưởng đến khả năng gửi tiền (`y=1`) | Mức độ ảnh hưởng  | Ghi chú                                                                    |
| ---------------------- | ------------------------------- | ---------------------------------- | --------------------------------------- | ----------------- | -------------------------------------------------------------------------- |
| **emp.var.rate**       | Tỷ lệ biến động việc làm        | Kinh tế ổn định hơn, việc làm tăng | 🔻 Giảm khả năng gửi tiền               | **Mạnh (âm)**     | Khi tỷ lệ việc làm cao, người dân chi tiêu nhiều hơn, gửi tiết kiệm ít hơn |
| **cons.price.idx**     | Chỉ số giá tiêu dùng (lạm phát) | Lạm phát tăng                      | 🔻 Giảm nhẹ khả năng gửi tiền           | **Yếu (âm)**      | Ảnh hưởng nhỏ, thể hiện qua chênh lệch nhỏ giữa hai nhóm                   |
| **cons.conf.idx**      | Niềm tin người tiêu dùng        | Tâm lý tiêu dùng tích cực hơn      | ⚪ Gần như không ảnh hưởng               | **Rất yếu**       | Phân bố hai nhóm gần như trùng nhau                                        |
| **euribor3m**          | Lãi suất Euribor 3 tháng        | Lãi suất thị trường tăng           | 🔻 Giảm mạnh khả năng gửi tiền          | **Mạnh (âm)**     | Khi lãi suất cao, khách hàng ưu tiên đầu tư hơn là gửi tiết kiệm           |
| **nr.employed**        | Số lượng người có việc làm      | Việc làm nhiều hơn                 | 🔻 Giảm khả năng gửi tiền               | **Khá mạnh (âm)** | Phản ánh mối liên hệ giữa thị trường lao động và hành vi tiết kiệm         |


## ⚖️ 3️⃣ XỬ LÝ MẤT CÂN BẰNG DỮ LIỆU
Dữ liệu bị **mất cân bằng** khi tỷ lệ khách hàng gửi tiền (`y=1`) rất nhỏ so với `y=0`.  
Ta dùng **SMOTE (Synthetic Minority Oversampling Technique)** để tạo thêm mẫu thiểu số nhân tạo.  
- Ưu điểm: Giữ lại toàn bộ dữ liệu gốc, tránh mất thông tin.  
- Nhược điểm: Có thể tạo nhiễu nếu dữ liệu thiểu số không phân bố tốt.


In [15]:
# Kiểm tra tỷ lệ nhãn
data.groupBy("y").count().show()

# Cân bằng dữ liệu (undersampling)
yes_count = data.filter(col("y") == "yes").count()
no_df = data.filter(col("y") == "no").sample(withReplacement=False, fraction=yes_count / data.filter(col("y") == "no").count())
yes_df = data.filter(col("y") == "yes")

data_balanced = no_df.union(yes_df)
print("Dữ liệu sau cân bằng:")
data_balanced.groupBy("y").count().show()


+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+

Dữ liệu sau cân bằng:
+---+-----+
|  y|count|
+---+-----+
| no| 4617|
|yes| 4640|
+---+-----+



## 🤖 4️⃣ XÂY DỰNG MÔ HÌNH CƠ BẢN
Ta bắt đầu bằng các **mô hình cơ bản** để thiết lập baseline:  
1. **Logistic Regression** – đơn giản, dễ giải thích, dùng làm baseline.  
2. **Random Forest** – bắt được quan hệ phi tuyến giữa các biến.  
3. **Gradient Boosted Trees (GBT)** – học sâu hơn mối quan hệ vĩ mô.  

Việc đánh giá dùng **Cross-validation (3-fold)** và chỉ số **Precision, Recall, F1-score**.


In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

# Đổi tên để tránh lỗi ký tự "."
data_balanced = (data_balanced
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

# Vector features
assembler = VectorAssembler(
    inputCols=["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"],
    outputCol="features"
)
final_data = assembler.transform(data_balanced).select("features", "y")

# Encode nhãn (PySpark yêu cầu dạng số)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="y", outputCol="label")
final_data = indexer.fit(final_data).transform(final_data)

train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)


In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import sum as _sum

models = {
    "Logistic Regression": LogisticRegression(featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50),
    "Gradient Boosted Trees": GBTClassifier(featuresCol="features", labelCol="label")
}

def compute_metrics(df):
    TP = df.filter((col("label") == 1) & (col("prediction") == 1)).count()
    FP = df.filter((col("label") == 0) & (col("prediction") == 1)).count()
    FN = df.filter((col("label") == 1) & (col("prediction") == 0)).count()
    TN = df.filter((col("label") == 0) & (col("prediction") == 0)).count()
    
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

results = []

for name, model in models.items():
    print(f"\n=== Mô hình: {name} ===")
    model_fit = model.fit(train_data)
    pred = model_fit.transform(test_data)
    
    precision, recall, f1 = compute_metrics(pred)
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    results.append((name, precision, recall, f1))



=== Mô hình: Logistic Regression ===
Precision: 0.6939, Recall: 0.7173, F1: 0.7054

=== Mô hình: Random Forest ===
Precision: 0.6826, Recall: 0.8772, F1: 0.7677

=== Mô hình: Gradient Boosted Trees ===
Precision: 0.6867, Recall: 0.8227, F1: 0.7486


## 📊 5️⃣ KẾT QUẢ ĐÁNH GIÁ


In [18]:
feature_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]
importance_summary = []

for name, model in models.items():
    fitted = model.fit(train_data)
    if hasattr(fitted, "featureImportances"):
        importances = list(fitted.featureImportances)
    elif hasattr(fitted, "coefficients"):
        importances = [abs(x) for x in fitted.coefficients]
    else:
        importances = [0.0]*len(feature_cols)
    
    pairs = list(zip(feature_cols, importances))
    print(f"\n=== Độ quan trọng ({name}) ===")
    for col_name, imp in pairs:
        print(f"{col_name:<20}: {imp:.6f}")
    
    max_idx = importances.index(max(importances, key=abs))
    importance_summary.append((name, feature_cols[max_idx], importances[max_idx]))

# Tổng kết
print("\n=== Tổng kết biến quan trọng nhất ===")
print(f"{'Mô hình':<30} {'Biến quan trọng nhất':<25} {'Giá trị':>10}")
for row in importance_summary:
    print(f"{row[0]:<30} {row[1]:<25} {row[2]:>10.6f}")



=== Độ quan trọng (Logistic Regression) ===
emp_var_rate        : 0.352835
cons_price_idx      : 0.367986
cons_conf_idx       : 0.022573
euribor3m           : 0.077800
nr_employed         : 0.008766

=== Độ quan trọng (Random Forest) ===
emp_var_rate        : 0.274023
cons_price_idx      : 0.016082
cons_conf_idx       : 0.064559
euribor3m           : 0.119589
nr_employed         : 0.525747

=== Độ quan trọng (Gradient Boosted Trees) ===
emp_var_rate        : 0.126242
cons_price_idx      : 0.086248
cons_conf_idx       : 0.061508
euribor3m           : 0.149200
nr_employed         : 0.576801

=== Tổng kết biến quan trọng nhất ===
Mô hình                        Biến quan trọng nhất         Giá trị
Logistic Regression            cons_price_idx              0.367986
Random Forest                  nr_employed                 0.525747
Gradient Boosted Trees         nr_employed                 0.576801
